# Homework 5

## Dictionaries

In [2]:
input = {'District_Haz': 1, 'Exc_Haz': 1, 'De_Haz': 1, 'Li_Haz': 1}
output = {'Leak_Haz': 1}

## Feature engineering

In [3]:
# Standard libraries:
import inspect
from time import time

# Third-party libraries:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, \
                            mean_squared_error, roc_auc_score
from sklearn.model_selection import train_test_split

In [4]:
%%time
# Import data with three unused columns dropped; split into train and test data; preview
df = pd.read_csv("ED-2_HW4.csv")
df, df_test = train_test_split(df, test_size=0.25, random_state = 1)
df.head()

Wall time: 252 ms


,Ag Bg,Condition,DIMP Material for Analysis,DIMP Wrap,Dimpcp,DIMPCause1,DIMPCause2,DIMPCause3,DIMP Pressure,Damaging Equipment,...,Leak Code for DOT,Leak Code for Reporting,Leak In,Material,Pipe Size,Region,Reporting Year,Riser Type,Type of Excavator,Type Repair
23842,BG,---,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Hand Tools,...,Hazardous,Code 1,Pipe,Plastic - Polyethylene,"1/2"" CTS",1,2015,NaN,Contractor,Altered Service - Immediate
26978,BG,NaN,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Unknown,Medium Pressure,NaN,...,Hazardous,Code 1,Pipe,Plastic - Polyethylene,"1/2"" CTS",4,2018,NP,NaN,Altered Service - Immediate
26056,BG,---,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Unknown/Other,...,Hazardous,Code 1,Pipe,Plastic - Polyethylene,"1/2"" IPS",4,2017,NaN,Contractor,Altered Service - Immediate
6332,BG,NaN,Steel,Coated (Condition Unknown),No CP,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Backhoe/Trackhoe,...,Hazardous,Code 1,Pipe,Steel - Coated w/o CP,"3/4""",3,2013,NaN,Contractor,Replace service
8755,BG,---,Steel,Coated (Condition Unknown),CP (Unknown Source),Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Backhoe/Trackhoe,...,Hazardous,Code 1,Other,Steel - Coated w/CP,"8""",1,2016,NaN,Contractor,Clamped


In [5]:
%%time
# Notice all but few are categorial variables
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20970 entries, 23842 to 235
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Ag Bg                       20970 non-null  object
 1   Condition                   10276 non-null  object
 2   DIMP Material for Analysis  20970 non-null  object
 3   DIMP Wrap                   20970 non-null  object
 4   Dimpcp                      20970 non-null  object
 5   DIMPCause1                  20970 non-null  object
 6   DIMPCause2                  20970 non-null  object
 7   DIMPCause3                  20954 non-null  object
 8   DIMP Pressure               20970 non-null  object
 9   Damaging Equipment          13315 non-null  object
 10  District                    20970 non-null  int64 
 11  EIR_NA                      20970 non-null  int64 
 12  Facility1                   20970 non-null  object
 13  Facility2                   20970 non-null  

In [6]:
%%time
# Rename column for usability; preview
df = df.rename(columns = {"Leak Code for Reporting" : "LeakCode"})
df = df.rename(columns = {"Type of Excavator" : "ToE"})
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns

Wall time: 14 ms


Index(['Ag_Bg', 'Condition', 'DIMP_Material_for_Analysis', 'DIMP_Wrap',
       'Dimpcp', 'DIMPCause1', 'DIMPCause2', 'DIMPCause3', 'DIMP_Pressure',
       'Damaging_Equipment', 'District', 'EIR_NA', 'Facility1', 'Facility2',
       'Facility3', 'Facility4', 'Fitting_Type', 'GO-112F_Group',
       'Investigation_Reason', 'Joint_Type', 'Key_Observations',
       'Leak_Code_for_DOT', 'LeakCode', 'Leak_In', 'Material', 'Pipe_Size',
       'Region', 'Reporting_Year', 'Riser_Type', 'ToE', 'Type_Repair'],
      dtype='object')

In [7]:
df = pd.DataFrame(df) 
df.EIR_NA.unique() 

array([0, 1], dtype=int64)

In [8]:
df['ToE'].value_counts().head()

Contractor                7968
Contractor - 3rd Party    3448
Occupant                  1453
Unknown                    538
Homeowner                  295
Name: ToE, dtype: int64

In [9]:
df['District'].value_counts().head()

56    878
38    859
50    842
24    818
3     802
Name: District, dtype: int64

In [10]:
df['Damaging_Equipment'].value_counts().head()

Hand Tools                                                                                                     5044
Backhoe/Trackhoe                                                                                               3635
Hand Tools including picks, shovels, bars, hand post diggers, similar equipment (no powered equipment) (HT)    1117
Backhoe/Trackhoe including backhoes, trackhoes, Gradalls, similar equipment (BT)                                977
Unknown/Other                                                                                                   541
Name: Damaging_Equipment, dtype: int64

In [11]:
df['Leak_In'].value_counts().head()

Pipe                    16660
Anodeless Riser           332
Riser (Above Ground)      290
Other                     173
Fitting                   123
Name: Leak_In, dtype: int64

In [12]:
%%time
# Create outcome variable to identify hazardous vs. non-hazardous leaks
def is_hazardous_leak(LeakCode):
    """Fcn to return if leak is hazardous.
       
       Arguments:
           - 'Leak Code for Reporting':   describes severity of leak
       
       Returns:
           - 1 if hazardous, 0 if non-hazardous
    """
    count = 0
    if (LeakCode == 'AG Hazardous') | (LeakCode == 'Code 1'):
        # If leak code is AG Haz or Code 1:
        count += 1
    return count

Wall time: 0 ns


In [13]:
%%time
# Create binary predictor using district to use in improved logistic regression model
def is_haz_district(District):
    """Fcn to return 1 if District is within the top five districts known to have excavation damage
    
       Returns:
           - 1 if in top 5 districts, 0 if not
    """
    count = 0
    if (District == '3') | (District == '50') | (District == '38') | (District == '56') | (District == '24'):
        count += 1
    return count

Wall time: 0 ns


In [14]:
%%time
# Create binary predictor using type of excavator to use in improved logistic regression model
def is_haz_exc(ToE):
    """Fcn to return 1 if excavator is a contractor
    
       Returns:
           - 1 if contractor, 0 if not
    """
    count = 0
    if (ToE == 'Contractor') | (ToE == 'Contractor - 3rd Party'):
        count += 1
    return count

Wall time: 0 ns


In [15]:
%%time
# Create binary predictor using damaging equipment to use in improved logistic regression model
def is_haz_de(Damaging_Equipment):
    """Fcn to return 1 if damaging equipment is a handtool 
    
       Returns:
           - 1 if handtool, 0 if not
    """
    count = 0
    if (Damaging_Equipment == 'Hand Tools'):
        count += 1
    return count

Wall time: 0 ns


In [16]:
%%time
# Create binary predictor using the part of the asset that leaked to use in improved logistic regression model
def is_haz_li(Leak_In):
    """Fcn to return 1 if leak occurred on a piece of pipe
    
       Returns:
           - 1 if pipe, 0 if not
    """
    count = 0
    if (Leak_In == 'Pipe'):
        count += 1
    return count

Wall time: 0 ns


In [17]:
%%time
# Add to data; preview
df['Leak_Haz'] = df[['LeakCode']].apply(lambda row: is_hazardous_leak(row[0]), axis = 1)
df['District_Haz'] = df[['District']].apply(lambda row: is_haz_district(row[0]), axis = 1)
df['Exc_Haz'] = df[['ToE']].apply(lambda row: is_haz_exc(row[0]), axis = 1)
df['De_Haz'] = df[['Damaging_Equipment']].apply(lambda row: is_haz_de(row[0]), axis = 1)
df['Li_Haz'] = df[['Leak_In']].apply(lambda row: is_haz_li(row[0]), axis = 1)
df.head()

Wall time: 1.04 s


,Ag_Bg,Condition,DIMP_Material_for_Analysis,DIMP_Wrap,Dimpcp,DIMPCause1,DIMPCause2,DIMPCause3,DIMP_Pressure,Damaging_Equipment,...,Region,Reporting_Year,Riser_Type,ToE,Type_Repair,Leak_Haz,District_Haz,Exc_Haz,De_Haz,Li_Haz
23842,BG,---,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Hand Tools,...,1,2015,NaN,Contractor,Altered Service - Immediate,1,0,1,1,1
26978,BG,NaN,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Unknown,Medium Pressure,NaN,...,4,2018,NP,NaN,Altered Service - Immediate,1,0,0,0,1
26056,BG,---,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Unknown/Other,...,4,2017,NaN,Contractor,Altered Service - Immediate,1,0,1,0,1
6332,BG,NaN,Steel,Coated (Condition Unknown),No CP,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Backhoe/Trackhoe,...,3,2013,NaN,Contractor,Replace service,1,0,1,0,1
8755,BG,---,Steel,Coated (Condition Unknown),CP (Unknown Source),Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Backhoe/Trackhoe,...,1,2016,NaN,Contractor,Clamped,1,0,1,0,0


In [18]:
df.tail()

,Ag_Bg,Condition,DIMP_Material_for_Analysis,DIMP_Wrap,Dimpcp,DIMPCause1,DIMPCause2,DIMPCause3,DIMP_Pressure,Damaging_Equipment,...,Region,Reporting_Year,Riser_Type,ToE,Type_Repair,Leak_Haz,District_Haz,Exc_Haz,De_Haz,Li_Haz
10955,BG,---,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Unknown,Medium Pressure,NaN,...,4,2019,NaN,Municipality,Altered Service - Immediate,1,0,0,0,1
17289,BG,---,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Data Not Collected,...,3,2017,NaN,Contractor,Other,1,0,1,0,1
5192,BG,NaN,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Homeowner,Medium Pressure,Hand Tools,...,4,2013,NaN,Homeowner,Altered,1,0,0,1,1
12172,BG,NaN,Steel,Coated (Condition Unknown),CP (Unknown Source),Excavation Damage,Excavation Damage by Third Party,Contractor - 3rd Party,Medium Pressure,Miscellaneous/Other including items not specif...,...,1,2018,NP,Contractor - 3rd Party,Altered Service - Immediate,1,0,1,0,1
235,BG,---,PE,Not Populated,Not Populated,Excavation Damage,Excavation Damage by Third Party,Contractor,Medium Pressure,Backhoe/Trackhoe,...,1,2015,NaN,Contractor,Altered Service - Immediate,1,0,1,0,1


In [19]:
%%time
# Majority of leak with excavation damage results in a hazardous leak
df['Leak_Haz'].value_counts()

Wall time: 1.99 ms


1    20671
0      299
Name: Leak_Haz, dtype: int64

In [20]:
%%time
# Convert categorical variables to integers to pass through model
df['Ag_Bg'] = df['Ag_Bg'].astype('category')
df['Ag_Bg'] = df['Ag_Bg'].cat.codes

df['Condition'] = df['Condition'].astype('category')
df['Condition'] = df['Condition'].cat.codes

df['Damaging_Equipment'] = df['Damaging_Equipment'].astype('category')
df['Damaging_Equipment'] = df['Damaging_Equipment'].cat.codes

df['DIMPCause1'] = df['DIMPCause1'].astype('category')
df['DIMPCause1'] = df['DIMPCause1'].cat.codes

df['DIMPCause2'] = df['DIMPCause2'].astype('category')
df['DIMPCause2'] = df['DIMPCause2'].cat.codes

df['DIMPCause3'] = df['DIMPCause3'].astype('category')
df['DIMPCause3'] = df['DIMPCause3'].cat.codes

df['Dimpcp'] = df['Dimpcp'].astype('category')
df['Dimpcp'] = df['Dimpcp'].cat.codes

df['District'] = df['District'].astype('category')
df['District'] = df['District'].cat.codes

df['DIMP_Material_for_Analysis'] = df['DIMP_Material_for_Analysis'].astype('category')
df['DIMP_Material_for_Analysis'] = df['DIMP_Material_for_Analysis'].cat.codes

df['DIMP_Pressure'] = df['DIMP_Pressure'].astype('category')
df['DIMP_Pressure'] = df['DIMP_Pressure'].cat.codes

df['DIMP_Wrap'] = df['DIMP_Wrap'].astype('category')
df['DIMP_Wrap'] = df['DIMP_Wrap'].cat.codes

df['EIR_NA'] = df['EIR_NA'].astype('category')
df['EIR_NA'] = df['EIR_NA'].cat.codes

df['Facility1'] = df['Facility1'].astype('category')
df['Facility1'] = df['Facility1'].cat.codes

df['Facility2'] = df['Facility2'].astype('category')
df['Facility2'] = df['Facility2'].cat.codes

df['Facility3'] = df['Facility3'].astype('category')
df['Facility3'] = df['Facility3'].cat.codes

df['Facility4'] = df['Facility4'].astype('category')
df['Facility4'] = df['Facility4'].cat.codes

df['Fitting_Type'] = df['Fitting_Type'].astype('category')
df['Fitting_Type'] = df['Fitting_Type'].cat.codes

df['GO-112F_Group'] = df['GO-112F_Group'].astype('category')
df['GO-112F_Group'] = df['GO-112F_Group'].cat.codes

df['Investigation_Reason'] = df['Investigation_Reason'].astype('category')
df['Investigation_Reason'] = df['Investigation_Reason'].cat.codes

df['Joint_Type'] = df['Joint_Type'].astype('category')
df['Joint_Type'] = df['Joint_Type'].cat.codes

df['Key_Observations'] = df['Key_Observations'].astype('category')
df['Key_Observations'] = df['Key_Observations'].cat.codes

df['LeakCode'] = df['LeakCode'].astype('category')
df['LeakCode'] = df['LeakCode'].cat.codes

df['Leak_Code_for_DOT'] = df['Leak_Code_for_DOT'].astype('category')
df['Leak_Code_for_DOT'] = df['Leak_Code_for_DOT'].cat.codes

df['Leak_In'] = df['Leak_In'].astype('category')
df['Leak_In'] = df['Leak_In'].cat.codes

df['Material'] = df['Material'].astype('category')
df['Material'] = df['Material'].cat.codes

df['Pipe_Size'] = df['Pipe_Size'].astype('category')
df['Pipe_Size'] = df['Pipe_Size'].cat.codes

df['Region'] = df['Region'].astype('category')
df['Region'] = df['Region'].cat.codes

df['Reporting_Year'] = df['Reporting_Year'].astype('category')
df['Reporting_Year'] = df['Reporting_Year'].cat.codes

df['Riser_Type'] = df['Riser_Type'].astype('category')
df['Riser_Type'] = df['Riser_Type'].cat.codes

df['ToE'] = df['ToE'].astype('category')
df['ToE'] = df['ToE'].cat.codes

df['Type_Repair'] = df['Type_Repair'].astype('category')
df['Type_Repair'] = df['Type_Repair'].cat.codes

df.head()

Wall time: 113 ms


,Ag_Bg,Condition,DIMP_Material_for_Analysis,DIMP_Wrap,Dimpcp,DIMPCause1,DIMPCause2,DIMPCause3,DIMP_Pressure,Damaging_Equipment,...,Region,Reporting_Year,Riser_Type,ToE,Type_Repair,Leak_Haz,District_Haz,Exc_Haz,De_Haz,Li_Haz
23842,1,0,0,4,4,0,10,3,2,16,...,0,4,-1,3,20,1,0,1,1,1
26978,1,-1,0,4,4,0,10,27,2,-1,...,3,7,2,-1,20,1,0,0,0,1
26056,1,0,0,4,4,0,10,3,2,25,...,3,6,-1,3,20,1,0,1,0,1
6332,1,-1,1,1,3,0,10,3,2,4,...,2,2,-1,3,95,1,0,1,0,1
8755,1,0,1,1,0,0,10,3,2,4,...,0,5,-1,3,46,1,0,1,0,0


## Final Model with Logistic Regression

In [21]:
y = df['Leak_Haz']
X = df[['EIR_NA', 'District_Haz', 'Exc_Haz', 'De_Haz']]
#X = df.drop(columns = ['Leak_Haz', 'Cause', 'Detect Date', 'Leak Fix Date', 'Key Observations', 'Leak Code for DOT', 'LeakCode'])

In [22]:
X.shape

(20970, 4)

In [23]:
inspect.signature(LogisticRegression)

<Signature (penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)>

In [24]:
est_model = LogisticRegression(penalty = 'none',
                              C = 0.5,
                              fit_intercept = True,
                              class_weight = 'balanced',
                              random_state = 1,
                              max_iter = 10000,
                              solver = 'lbfgs')

In [25]:
est_model

LogisticRegression(C=0.5, class_weight='balanced', max_iter=10000,
                   penalty='none', random_state=1)

In [26]:
est_model.fit(X, y)

C:\Users\airit\Anaconda3\envs\env-stats404-w21\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


LogisticRegression(C=0.5, class_weight='balanced', max_iter=10000,
                   penalty='none', random_state=1)

In [27]:
coefficients = [round(x, 2) for x in est_model.coef_.tolist()[0]]
coef_df = pd.concat([pd.DataFrame(X.columns),
                     pd.DataFrame(coefficients)],
                    axis = 1)
coef_df.columns = ["feature_name", "coef_est"]

# DataFrame of intercept coefficient:
intercept_df = pd.DataFrame(["intercept", est_model.intercept_.tolist()[0]]).T
intercept_df.columns = ["feature_name", "coef_est"]

# Combined DataFrame:
coef_df = coef_df.append(intercept_df)
coef_df.head()

,feature_name,coef_est
0,EIR_NA,-1.51
1,District_Haz,0.0
2,Exc_Haz,0.3
3,De_Haz,-0.06
0,intercept,0.526299


In [28]:
# Compute odds:
coef_df["odds"] = [round(np.exp(x), 2) for x in coef_df['coef_est']]

In [29]:
# Compute probabilities, which need intercept:
intercept = intercept_df['coef_est'].values.tolist()[0]
intercept

0.526299489557078

In [30]:
def inverse_logit(intercept, coefficient):
    """Fcn to help calculate probability associated with flight delay,
       given our variable.
    """
    if coefficient == intercept:
        # Make sure we don't double-count the intercept:
        coefficient = 0
    inv_logit = np.exp(intercept + coefficient)/(1+np.exp(intercept + coefficient))
    return round(inv_logit, 2)
                           
coef_df["prob"] = [inverse_logit(intercept, x) for x in coef_df['coef_est']]

In [31]:
coef_df.T

,0,1,2,3,0
feature_name,EIR_NA,District_Haz,Exc_Haz,De_Haz,intercept
coef_est,-1.51,0.0,0.3,-0.06,0.526299
odds,0.22,1.0,1.35,0.94,1.69
prob,0.27,0.63,0.7,0.61,0.63


In [32]:
y_pred_train = est_model.predict(X)
y_pred_train[0:5]

array([1, 0, 1, 1, 1], dtype=int64)

In [33]:
y_pred_train_probs = pd.DataFrame(est_model.predict_proba(X))
y_pred_train_probs.head()

,0,1
0,0.316311,0.683689
1,0.727578,0.272422
2,0.303536,0.696464
3,0.303536,0.696464
4,0.303536,0.696464


In [34]:
inspect.signature(confusion_matrix)

<Signature (y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)>

In [35]:
confusion_matrix(y_true=y,
                 y_pred=y_pred_train)

array([[  187,   112],
       [ 4842, 15829]], dtype=int64)

In [36]:
inspect.signature(roc_auc_score)

<Signature (y_true, y_score, *, average='macro', sample_weight=None, max_fpr=None, multi_class='raise', labels=None)>

In [37]:
roc_auc_score(y_true=y,
              y_score=y_pred_train_probs.iloc[:, 1])

0.7084621160726522

In [42]:
est_model.score(X, y)

0.7637577491654745